In [1]:
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright

In [2]:
url = 'https://assist.org/transfer/results?year=75&institution=79&agreement=124&agreementType=from&viewAgreementsOptions=true&view=agreement&viewBy=major&viewSendingAgreements=false&viewByKey=75%2F124%2Fto%2F79%2FMajor%2F23d79a84-d16c-4b58-7dee-08dcb87d5deb'

In [3]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=True)
page = await browser.new_page()
await page.goto(url, wait_until='networkidle')
await page.wait_for_selector('.rowReceiving')

<JSHandle preview=JSHandle@node>

In [4]:
content = await page.content()
soup = BeautifulSoup(content, 'lxml')
courses = soup.find_all('div', class_='rowReceiving')
print(f"Found {len(courses)} receiving course blocks.")

Found 11 receiving course blocks.


In [ ]:
attributes = soup.find_all(class_='attributeContent')

In [9]:
for attribute in attributes:
    print(' '.join(attribute.stripped_strings))

Recommended to be completed prior to transfer
Please refer to additional important General Information section above
Regular and honors courses may be combined to complete this series
Regular and honors courses may be combined to complete this series
Same as ENVECON C1
Same as ECON 6
Please refer to additional important General Information section above
Same as COMPSCI C8, INFO C8, STAT C8
Same as COMPSCI C88C
